## 1. Import Necessary Libraries

In [35]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import metrics
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau


## 2. Load Data + Data Preprocessing

In [37]:
# Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')
print(train_df.shape[0])

# Generate file paths correctly
train_df['file_path'] = train_df.apply(
    lambda row: f"/kaggle/input/bttai-ajl-2025/train/train/{row['label']}/{row['md5hash']}.jpg", axis=1
)
test_df['file_path'] = test_df['md5hash'].apply(
    lambda x: f"/kaggle/input/bttai-ajl-2025/test/test/{x}.jpg"
)

# Data Preprocessing

# Remove invalid rows
train_df = train_df[(train_df['fitzpatrick_scale'] > 0) & (train_df['label'].notna())]
print(train_df.shape[0])

train_df = train_df[train_df['file_path'].apply(os.path.exists)]
print(train_df.shape[0])

test_df = test_df[test_df['file_path'].apply(os.path.exists)]
print(test_df.shape[0])
print()


# Encode the labels
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])


# Splitting dataset into training and validation datasets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['encoded_label'])

# Define image data generators for training and testing
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.9, 1.1],
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=512,
    class_mode='raw',
    shuffle = True
)


val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=512,
    class_mode='raw',
    shuffle=False
    
)


test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='file_path',
    target_size=(224, 224),
    batch_size= 512,
    class_mode=None,
    shuffle=False
    
)


2860
2752
2752
1227

Found 2201 validated image filenames.
Found 551 validated image filenames.
Found 1227 validated image filenames.


In [38]:
# Common Model Parameters
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df['encoded_label']),
    y=train_df['encoded_label']
)
class_weights_dict = dict(enumerate(class_weights))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Initialize the ReduceLROnPlateau callback
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 factor=0.5,  # Factor to reduce the learning rate
                                 patience=3,  # Number of epochs to wait before reducing
                                 min_lr=1e-6)  # Minimum learning rate

## 4. Model Training

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
for layer in base_model.layers[:230]:  
    layer.trainable = False  



efficientnetb0 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),
    layers.Dropout(0.25),
    layers.Dense(21, activation='softmax')
])

efficientnetb0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

efficientnetb0.fit(train_generator, epochs=30, validation_data=val_generator, class_weight = class_weights_dict callbacks=[lr_reduction])


In [43]:
# Generate predictions
# y_prob_res = ResNet101.predict(val_generator)
y_prob_eff = efficientnetb0.predict(val_generator)
# y_prob_combined = (0.6 * y_prob_res) + (0.4 * y_prob_eff)


# y_pred_res = np.argmax(y_prob_res, axis=1) 
y_pred_eff = np.argmax(y_prob_eff, axis=1)
# y_pred_combined = np.argmax(y_prob_combined, axis=1)


y_true = val_df['encoded_label'].values 

# Calculate F1 Score
# f1 = f1_score(y_true, y_pred_res, average='weighted')
# print("ResNet 101 F1 Score:", f1)

f1 = f1_score(y_true, y_pred_eff, average='weighted')
print("Efficient Net b0 F1 Score:", f1)

# f1 = f1_score(y_true, y_pred_combined, average='weighted')
# print("Combined F1 Sccore", f1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 5s/step
Efficient Net b0 F1 Score: 0.4473521202175346


In [41]:
# SUBMISSION.CSV
y_pred = np.argmax(efficientnetb0.predict(test_generator), axis = 1)
test_df['label'] = label_encoder.inverse_transform(y_pred)

# Save submission
test_df[['md5hash', 'label']].to_csv('/kaggle/working/submission.csv', index=False)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 25s 9s/step 
